In [ ]:
import os
import io
import IPython.display
from PIL import Image
import base64 
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
import gradio as gr

In [ ]:
#import torch

#torch.cuda.is_available()


#torch.cuda.device_count()


#torch.cuda.current_device()


#torch.cuda.device(0)


#torch.cuda.get_device_name(0)

# TEXT SUMMARIZATION

In [ ]:
from transformers import pipeline

get_completion = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device=0)

def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']

In [ ]:
import gradio as gr

def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']
    
gr.close_all()
demo = gr.Interface(fn=summarize, 
                    inputs=[gr.Textbox(label="Text to summarize", lines=6)],
                    outputs=[gr.Textbox(label="Result", lines=3)],
                    title="Text summarization with distilbart-cnn",
                    description="Summarize any text using the `shleifer/distilbart-cnn-12-6` model under the hood!"
                   )
demo.launch(server_name='0.0.0.0')

# NAMED ENTITY RECOGNITION

In [ ]:
from transformers import pipeline

get_completion = pipeline("ner", model="dslim/bert-base-NER", device=0)

def ner(input):
    output = get_completion(input)
    return {"text": input, "entities": output}

In [ ]:
def merge_tokens(tokens):
    merged_tokens = []
    for token in tokens:
        if merged_tokens and token['entity'].startswith('I-') and merged_tokens[-1]['entity'].endswith(token['entity'][2:]):
            # If current token continues the entity of the last one, merge them
            last_token = merged_tokens[-1]
            last_token['word'] += token['word'].replace('##', '')
            last_token['end'] = token['end']
            last_token['score'] = (last_token['score'] + token['score']) / 2
        else:
            # Otherwise, add the token to the list
            merged_tokens.append(token)

    return merged_tokens

def ner(input):
    output = get_completion(input)
    merged_tokens = merge_tokens(output)
    return {"text": input, "entities": merged_tokens}

gr.close_all()
demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
                    outputs=[gr.HighlightedText(label="Text with entities")],
                    title="NER with dslim/bert-base-NER",
                    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
                    allow_flagging="never",
                    examples=["My name is Andrew, I'm building DeeplearningAI and I live in California", "My name is Poli, I live in Vienna and work at HuggingFace"])

demo.launch(share=False, server_name='0.0.0.0')

# IMAGE CAPTIONING

In [ ]:
from transformers import pipeline

get_completion = pipeline("image-to-text",model="Salesforce/blip-image-captioning-base", device=0)

def summarize(input):
    output = get_completion(input)
    return output[0]['generated_text']

In [ ]:
image_url = "https://free-images.com/sm/9596/dog_animal_greyhound_983023.jpg"
display(IPython.display.Image("carro.png"))
get_completion("carro.png")

In [ ]:
import gradio as gr 

# def image_to_base64_str(pil_image):
#     byte_arr = io.BytesIO()
#     pil_image.save(byte_arr, format='PNG')
#     byte_arr = byte_arr.getvalue()
#     return str(base64.b64encode(byte_arr).decode('utf-8'))

def captioner(image):
    result = get_completion(image)
    return result[0]['generated_text']

gr.close_all()
demo = gr.Interface(fn=captioner,
                    inputs=[gr.Image(label="Upload image", type="pil")],
                    outputs=[gr.Textbox(label="Caption")],
                    title="Image Captioning with BLIP",
                    description="Caption any image using the BLIP model",
                    allow_flagging="never",
                    #examples=["christmas_dog.jpeg", "bird_flight.jpeg", "cow.jpeg"]
                    )

demo.launch(share=False, server_name='0.0.0.0')

# IMAGE GENERATION

In [ ]:
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")

def get_completion(prompt):
    return pipeline(prompt).images[0]

In [ ]:
prompt = "a dog in a park"

result = get_completion(prompt)
IPython.display.HTML(f'<img src="data:image/png;base64,{result}" />')

## Generating with `gr.Interface()`

In [ ]:
import gradio as gr 

#A helper function to convert the PIL image to base64
#so you can send it to the API
# def base64_to_pil(img_base64):
#     base64_decoded = base64.b64decode(img_base64)
#     byte_stream = io.BytesIO(base64_decoded)
#     pil_image = Image.open(byte_stream)
#     return pil_image

# def generate(prompt):
#     output = get_completion(prompt)
#     result_image = base64_to_pil(output)
#     return result_image

def generate(prompt):
    output = get_completion(prompt)
    result_image = output
    return result_image

gr.close_all()
demo = gr.Interface(fn=generate,
                    inputs=[gr.Textbox(label="Your prompt")],
                    outputs=[gr.Image(label="Result")],
                    title="Image Generation with Stable Diffusion",
                    description="Generate any image with Stable Diffusion",
                    allow_flagging="never",
                    examples=["the spirit of a tamagotchi wandering in the city of Vienna","a mecha robot in a favela"])

demo.launch(share=False, server_name='0.0.0.0', server_port='7860')

## Building a more advanced interface

In [ ]:
import gradio as gr 

#A helper function to convert the PIL image to base64 
# so you can send it to the API
# def base64_to_pil(img_base64):
#     base64_decoded = base64.b64decode(img_base64)
#     byte_stream = io.BytesIO(base64_decoded)
#     pil_image = Image.open(byte_stream)
#     return pil_image

# def generate(prompt, negative_prompt, steps, guidance, width, height):
#     params = {
#         "negative_prompt": negative_prompt,
#         "num_inference_steps": steps,
#         "guidance_scale": guidance,
#         "width": width,
#         "height": height
#     }
    
#     output = get_completion(prompt, params)
#     pil_image = base64_to_pil(output)
#     return pil_image

def generate(prompt, negative_prompt, steps, guidance, width, height):
    params = {
        "negative_prompt": negative_prompt,
        "num_inference_steps": steps,
        "guidance_scale": guidance,
        "width": width,
        "height": height
    }
    
    output = get_completion(prompt, params)
    pil_image = output
    return pil_image

gr.close_all()
demo = gr.Interface(fn=generate,
                    inputs=[
                        gr.Textbox(label="Your prompt"),
                        gr.Textbox(label="Negative prompt"),
                        gr.Slider(label="Inference Steps", minimum=1, maximum=100, value=25,
                                 info="In how many steps will the denoiser denoise the image?"),
                        gr.Slider(label="Guidance Scale", minimum=1, maximum=20, value=7, 
                                  info="Controls how much the text prompt influences the result"),
                        gr.Slider(label="Width", minimum=64, maximum=512, step=64, value=512),
                        gr.Slider(label="Height", minimum=64, maximum=512, step=64, value=512),
                    ],
                    outputs=[gr.Image(label="Result")],
                    title="Image Generation with Stable Diffusion",
                    description="Generate any image with Stable Diffusion",
                    allow_flagging="never"
                    )

demo.launch(share=True, server_port=int(os.environ['PORT2']))

# QUESTION ANSWERING IN PORTUGUESE

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("question-answering", model="mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt")

In [ ]:
import gradio as gr

def qna(input):
    output = pipe(input)
    return output[0]['resposta']
    
gr.close_all()
demo = gr.Interface(fn=qna, 
                    inputs=[gr.Textbox(label="Faça uma pergunta", lines=6)],
                    outputs=[gr.Textbox(label="Resultado", lines=3)],
                    title="Q&A com bert-base-portuguese-cased-finetuned-squad-v1-pt",
                    description="Perguntas e respostas com o modelo bert-base-portuguese-cased-finetuned-squad-v1-pt"
                   )
demo.launch(server_name='0.0.0.0')

In [ ]:
from transformers import AutoTokenizer, BertForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt")
model = BertForQuestionAnswering.from_pretrained("mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt")

question, text = "Qual é a estação do ano mais linda??", "verão, outono,inverno,primavera"

inputs = tokenizer(question, text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)

# ChatBot with Microsoft's DialoGPT

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

In [ ]:
def qna(input):
    output = pipe(input)
    return output[0]['resposta']
    
gr.close_all()
demo = gr.Interface(fn=qna, 
                    inputs=[gr.Textbox(label="Faça uma pergunta em inglês", lines=6)],
                    outputs=[gr.Textbox(label="Resultado", lines=3)],
                    title="Q&A com microsoft-dialogpt",
                    description="Perguntas e respostas com o modelo DialoGPT"
                   )
demo.launch(server_name='0.0.0.0')

In [ ]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from threading import Thread

tokenizer = AutoTokenizer.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-3B-v1")
model = AutoModelForCausalLM.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-3B-v1", torch_dtype=torch.float16)
model = model.to('cuda:0')

class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = [29, 0]
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

def predict(message, history):

    history_transformer_format = history + [[message, ""]]
    stop = StopOnTokens()

    messages = "".join(["".join(["\n<human>:"+item[0], "\n<bot>:"+item[1]])  #curr_system_message +
                for item in history_transformer_format])

    model_inputs = tokenizer([messages], return_tensors="pt").to("cuda")
    streamer = TextIteratorStreamer(tokenizer, timeout=10., skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = dict(
        model_inputs,
        streamer=streamer,
        max_new_tokens=1024,
        do_sample=True,
        top_p=0.95,
        top_k=1000,
        temperature=1.0,
        num_beams=1,
        stopping_criteria=StoppingCriteriaList([stop])
        )
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()

    partial_message  = ""
    for new_token in streamer:
        if new_token != '<':
            partial_message += new_token
            yield partial_message


gr.ChatInterface(predict).queue().launch()

In [ ]:
gr.close_all()